In [18]:
import pandas as pd
from pandas import Series
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.float_format', '{:.5f}'.format)

In [3]:
village_list = []
village = pd.read_csv("village.csv")
for i in village["village"]:
    village_list.append(i)
# village_list

In [4]:
train_data = pd.read_excel("train.xlsx")
train_data

,area,probability,total pepple,rent,tax,MRT,count_school
0,民安里 101,0.00000,4847,450000.00000,2136759,3,0
1,民安里 102,0.07140,4815,140833.33330,2098791,3,0
2,民安里 103,0.00000,4841,378363.60000,2147729,3,0
3,民安里 104,0.00000,4781,159800.00000,2091282,3,0
4,民安里 105,0.00000,4735,185882.12500,2106819,3,0
...,...,...,...,...,...,...,...
289,新庄里103,0.00000,5231,45000.00000,1523155,0,0
290,新庄里104,0.00000,5264,77375.00000,1454065,0,0
291,新庄里105,0.00000,5300,34333.33333,1503319,0,0
292,新庄里106,0.00000,6798,70000.00000,1448754,0,0


In [5]:
village_list_time = []
for i in train_data["area"]:
    village_list_time.append(i)
village_list_time

['民安里 101',
 '民安里 102',
 '民安里 103',
 '民安里 104',
 '民安里 105',
 '民安里 106',
 '民安里 107',
 '康樂里101',
 '康樂里102',
 '康樂里103',
 '康樂里104',
 '康樂里105',
 '康樂里106',
 '康樂里107',
 '中山里101',
 '中山里102',
 '中山里103',
 '中山里104',
 '中山里105',
 '中山里106',
 '中山里107',
 '聚盛里101',
 '聚盛里102',
 '聚盛里103',
 '聚盛里104',
 '聚盛里105',
 '聚盛里106',
 '聚盛里107',
 '正守里101',
 '正守里102',
 '正守里103',
 '正守里104',
 '正守里105',
 '正守里106',
 '正守里107',
 '正義里101',
 '正義里102',
 '正義里103',
 '正義里104',
 '正義里105',
 '正義里106',
 '正義里107',
 '正得里101',
 '正得里102',
 '正得里103',
 '正得里104',
 '正得里105',
 '正得里106',
 '正得里107',
 '集英里101',
 '集英里102',
 '集英里103',
 '集英里104',
 '集英里105',
 '集英里106',
 '集英里107',
 '聚葉里101',
 '聚葉里102',
 '聚葉里103',
 '聚葉里104',
 '聚葉里105',
 '聚葉里106',
 '聚葉里107',
 '中吉里101',
 '中吉里102',
 '中吉里103',
 '中吉里104',
 '中吉里105',
 '中吉里106',
 '中吉里107',
 '中原里101',
 '中原里102',
 '中原里103',
 '中原里104',
 '中原里105',
 '中原里106',
 '中原里107',
 '興亞里101',
 '興亞里102',
 '興亞里103',
 '興亞里104',
 '興亞里105',
 '興亞里106',
 '興亞里107',
 '埤頭里101',
 '埤頭里102',
 '埤頭里103',
 '埤頭里104',
 '埤頭里105',
 '埤頭里106',
 '埤

In [6]:
train_data.index = village_list_time
train = train_data.drop(["area"], axis=1)
train

,probability,total pepple,rent,tax,MRT,count_school
民安里 101,0.00000,4847,450000.00000,2136759,3,0
民安里 102,0.07140,4815,140833.33330,2098791,3,0
民安里 103,0.00000,4841,378363.60000,2147729,3,0
民安里 104,0.00000,4781,159800.00000,2091282,3,0
民安里 105,0.00000,4735,185882.12500,2106819,3,0
...,...,...,...,...,...,...
新庄里103,0.00000,5231,45000.00000,1523155,0,0
新庄里104,0.00000,5264,77375.00000,1454065,0,0
新庄里105,0.00000,5300,34333.33333,1503319,0,0
新庄里106,0.00000,6798,70000.00000,1448754,0,0


## X_train

In [7]:
X_train = train.drop(["probability"], axis=1)
X_train

,total pepple,rent,tax,MRT,count_school
民安里 101,4847,450000.00000,2136759,3,0
民安里 102,4815,140833.33330,2098791,3,0
民安里 103,4841,378363.60000,2147729,3,0
民安里 104,4781,159800.00000,2091282,3,0
民安里 105,4735,185882.12500,2106819,3,0
...,...,...,...,...,...
新庄里103,5231,45000.00000,1523155,0,0
新庄里104,5264,77375.00000,1454065,0,0
新庄里105,5300,34333.33333,1503319,0,0
新庄里106,6798,70000.00000,1448754,0,0


## Y_train

In [8]:
col_list = ["probability"]
Y_train = train_data[col_list]
train_data = train_data.drop(["probability"], axis=1)
Y_train 

,probability
民安里 101,0.00000
民安里 102,0.07140
民安里 103,0.00000
民安里 104,0.00000
民安里 105,0.00000
...,...
新庄里103,0.00000
新庄里104,0.00000
新庄里105,0.00000
新庄里106,0.00000


In [9]:
test_data = pd.read_csv("test_data.csv")
test_data

,village,probability,total pepple,rent,tax,MRT,count_school
0,民安里,0.00000,4553,123830.57140,1754879,3,0
1,康樂里,0.08330,2613,7500.00000,943002,0,0
2,中山里,0.00000,6851,78900.00000,2393322,0,0
3,聚盛里,0.00000,4209,101190.71430,1361132,0,0
4,正守里,0.00000,3322,52500.00000,1236901,0,0
5,正義里,0.00000,5297,53285.71429,1380200,0,0
6,正得里,0.00000,3711,90000.00000,1674992,0,0
7,集英里,0.00000,7443,124872.00000,2521107,7,3
8,聚葉里,0.00000,4052,320000.00000,1297945,2,1
9,中吉里,0.08330,5891,44000.00000,2476087,1,0


# X_test

In [10]:
test_data.index = village_list
X_test = test_data.drop(["village", "probability"], axis=1)
X_test

,total pepple,rent,tax,MRT,count_school
民安里,4553,123830.57140,1754879,3,0
康樂里,2613,7500.00000,943002,0,0
中山里,6851,78900.00000,2393322,0,0
聚盛里,4209,101190.71430,1361132,0,0
正守里,3322,52500.00000,1236901,0,0
正義里,5297,53285.71429,1380200,0,0
正得里,3711,90000.00000,1674992,0,0
集英里,7443,124872.00000,2521107,7,3
聚葉里,4052,320000.00000,1297945,2,1
中吉里,5891,44000.00000,2476087,1,0


## Y_test

In [11]:
Y_test = test_data[col_list]
test_data = test_data.drop(["probability"], axis=1)
Y_test

,probability
民安里,0.00000
康樂里,0.08330
中山里,0.00000
聚盛里,0.00000
正守里,0.00000
正義里,0.00000
正得里,0.00000
集英里,0.00000
聚葉里,0.00000
中吉里,0.08330


In [12]:
scaler_train_x = MinMaxScaler()
X_train_norm = scaler_train_x.fit(X_train)
#X_train_norm
X_train_scaled = X_train_norm.transform(X_train)

X_test_norm = X_train_norm.transform(X_test)

scaler_train_y = MinMaxScaler()
Y_train_norm = scaler_train_y.fit(Y_train)

Y_train_scaled = Y_train_norm.transform(Y_train)

Y_test_norm = Y_train_norm.transform(Y_test)
#scaler_test_x = MinMaxScaler()
#X_test_norm = scaler_test_x.fit_transform(X_test)


In [19]:
GBR = GradientBoostingRegressor()
GBR

GradientBoostingRegressor()

In [21]:
parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
              'subsample'    : [0.9, 0.5, 0.2, 0.1],
              'n_estimators' : [100,500,1000, 1500],
              'max_depth'    : [4,6,8,10]
             }

In [23]:
grid_GBR = GridSearchCV(estimator=GBR, param_grid = parameters, cv = 2, n_jobs=-1)
grid_GBR.fit(X_train_scaled, Y_train_scaled)

C:\Users\Frank\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=2, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.02, 0.03, 0.04],
                         'max_depth': [4, 6, 8, 10],
                         'n_estimators': [100, 500, 1000, 1500],
                         'subsample': [0.9, 0.5, 0.2, 0.1]})

In [24]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",grid_GBR.best_estimator_)
print("\n The best score across ALL searched params:\n",grid_GBR.best_score_)
print("\n The best parameters across ALL searched params:\n",grid_GBR.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 GradientBoostingRegressor(learning_rate=0.01, max_depth=4, subsample=0.1)

 The best score across ALL searched params:
 -0.04127859719165128

 The best parameters across ALL searched params:
 {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.1}


In [14]:
other_params = {"learning_rate":0.01, "n_estimators":1000, "max_depth":3,
                "min_child_weight":2, "seed":0, "subsample":0.2, "colsample_bytree":0.8,
                "gamma":0, "reg_alpha":0, "reg_lambda":0}

In [15]:
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, **other_params).fit(X_train_scaled, Y_train_scaled)
y_pred = model.predict(X_test_norm)
y_train_pred = model.predict(X_train_scaled)
print(y_pred)
re_y_pred = Y_train_norm.inverse_transform(y_pred.reshape(-1,1))
print(re_y_pred)

[ 0.12534429  0.179578    0.0673417   0.01691435  0.02428245  0.08078755
  0.1495919  -0.02240247  0.0217765  -0.0121378   0.11075989  0.08359816
  0.01487773  0.0085609   0.12089393  0.00214579  0.02457717  0.08765499
  0.05330579  0.04578309  0.08335128  0.05372816 -0.0373738   0.06994367
  0.18979795 -0.03300139  0.09080274  0.02501178  0.06463007  0.05288928
  0.09598356  0.02169634  0.03248505  0.02409307  0.05805913  0.07914729
  0.01187058  0.00857767  0.10665149  0.04350031  0.09681378  0.07126689]
[[ 0.05372256]
 [ 0.07696714]
 [ 0.02886265]
 [ 0.00724949]
 [ 0.01040746]
 [ 0.03462555]
 [ 0.06411508]
 [-0.0096017 ]
 [ 0.00933341]
 [-0.00520226]
 [ 0.04747169]
 [ 0.03583017]
 [ 0.00637659]
 [ 0.0036692 ]
 [ 0.05181514]
 [ 0.00091968]
 [ 0.01053377]
 [ 0.03756893]
 [ 0.02284686]
 [ 0.01962263]
 [ 0.03572436]
 [ 0.02302789]
 [-0.01601841]
 [ 0.02997786]
 [ 0.08134741]
 [-0.01414439]
 [ 0.03891805]
 [ 0.01072005]
 [ 0.02770045]
 [ 0.02266834]
 [ 0.04113855]
 [ 0.00929905]
 [ 0.013

In [16]:
y_pred = pd.DataFrame(y_pred, columns=["output"],dtype=float)
y_pred

,output
0,0.12534
1,0.17958
2,0.06734
3,0.01691
4,0.02428
5,0.08079
6,0.14959
7,-0.02240
8,0.02178
9,-0.01214


In [17]:
mae = mean_absolute_error(Y_test, re_y_pred)
mse = mean_squared_error(Y_test, re_y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(Y_train, y_train_pred)
# rmsle = np.sqrt(mean_squared_log_error(Y_test, y_pred))
# mape = np.mean(np.abs((Y_test, y_pred) /Y_test)) * 100
print(f"mae : {mae}")
print(f"mse : {mse}")
print(f"rmse : {rmse}")
print(f"r2 : {r2}")
# print(f"rmsle : {rmsle}")
# print(f"mape : {mape}")

mae : 0.0423636669198672
mse : 0.003708537394880667
rmse : 0.06089776182160283
r2 : 0.10890250596101514
